In [1]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", torch_dtype=torch.float32)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [73]:

prompt = """Complete the following Shakespearean sonnet starting from the first line:

FROM fairest creatures we desire increase,"""


prompt_Romana = """Completează următorul sonet în stilul lui Mihai Eminescu, începând cu primul vers:

Când însuși glasul gândurilor tace,"""


prompt_Pastel_Nature = """Write verses in a calm and beautiful pastoral style, describing a peaceful spring morning in the countryside. Focus on the beauty of nature—flowers blooming, birds singing, and the gentle breeze. Use soft, vivid language to capture the tranquility of the scene. Each verse should evoke a sense of peace and harmony with the natural world.

Write the poem, generate the verses:
"""


inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_new_tokens=100, # daca ii mai mare genereaza text mai lung
    # 0.6 pentru prompt, 0.9 pentru pastel
    temperature=0.6, # cu cat ii mare - mai creativ,haotic :  mai mic - mai logic,repetitiv
    top_p=0.9, # cu cat ii mare cu cat ii mai creativ
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id, # previne generarea infinita
    num_return_sequences = 1 # numarul de variante generate
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Complete the following Shakespearean sonnet starting from the first line:

FROM fairest creatures we desire increase,

That thereby beauty's rose might never die,

But as the riper should by time decease,

His tender heir might bear his memory:

But thy perfection doubtless durst show,

If, for thy sake, we then have been

To bend to thee our bodies and our blood,

To which our souls are bound forever.

A. What does the speaker mean by the "world" 'in lines 2 and 3? B. What is the "


In [13]:

prompt_Romana = """Completează următorul sonet în stilul lui Mihai Eminescu, începând cu primul vers:

Când însuși glasul gândurilor tace,"""



inputs = tokenizer(prompt_Romana, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.6,
    top_k = 20,
    top_p=0.9,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    num_return_sequences = 1
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Completează următorul sonet în stilul lui Mihai Eminescu, începând cu primul vers:

Când însuși glasul gândurilor tace,

Când nu mai e nimic ca să se spargă

De-a mărturisit, de-a mărturisit

De-a mărturisit, de-a mărturisit

...

Când nu mai e nimic ca să se spargă

De-a mărturisit, de-a mărturisit

De-a mărturisit, de-a mărturisit

De-a mărturisit


<h3> LLM antrenat pe poezii: </h3>


In [3]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel,GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM
import tensorflow as tf

tokenizer2 = GPT2Tokenizer.from_pretrained("ashiqabdulkhader/GPT2-Poet")
model2 = TFGPT2LMHeadModel.from_pretrained("ashiqabdulkhader/GPT2-Poet")



All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ashiqabdulkhader/GPT2-Poet.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [75]:
text = "FROM fairest creatures we desire increase,"
text_Romana = "Când însuși glasul gândurilor tace"

text_Pastel = """Write verses in a calm and beautiful pastoral style, describing a peaceful spring morning in the countryside. Focus on the beauty of nature—flowers blooming, birds singing, and the gentle breeze. Use soft, vivid language to capture the tranquility of the scene. Each verse should evoke a sense of peace and harmony with the natural world.

Write the poem, generate the verses:
"""

# Codifică promptul
inputs = tokenizer2(text, return_tensors="tf")


sample_outputs = model2.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=True,
    # 100 pt text, 200 pastel
    max_length=100,
    top_k=0, # estimeaza probabilitati pt cuvinte si alege aleator doar primele n
    top_p=0.9,
    temperature=1.0,
    num_return_sequences=1
)

print("Output:", tokenizer2.decode(sample_outputs[0], skip_special_tokens=True))



Output: FROM fairest creatures we desire increase, and like stars. likewise the night i rode back to my little east (which i have not thought of love as i've pried in a play that i am trying to be the last half hour from the flower of the morning—but the still shadowy is that we once had still half buried beneath) for me a wonder for me has come, and the still voice of thy lips made pale in thy breath, and the still silent is taller now than


In [10]:
text_Romana = "Când însuși glasul gândurilor tace"


# Codifică promptul
inputs = tokenizer2(text_Romana, return_tensors="tf")


sample_outputs = model2.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=True,
    max_length=100,
    top_k=20,
    top_p=0.9,
    temperature=1.0,
    num_return_sequences=1
)

print("Output:", tokenizer2.decode(sample_outputs[0], skip_special_tokens=True))



Output: Când însuși glasul gândurilor tace. still, a girl,with no clue, saved at her, and kept back,i broke a girl. and now the mirror strikes,its long black pinwheel slack as she pins!the whole town, as ever, hearing, fall, its last look has time to follow.the stenchy, melancholy, melancholy, eternally steep,yet from the margin of her frame has


In [11]:
text_Pastel = """Write verses in a calm and beautiful pastoral style, describing a peaceful spring morning in the countryside. Focus on the beauty of nature—flowers blooming, birds singing, and the gentle breeze. Use soft, vivid language to capture the tranquility of the scene. Each verse should evoke a sense of peace and harmony with the natural world.

Write the poem, generate the verses:
"""

# Codifică promptul
inputs = tokenizer2(text_Pastel, return_tensors="tf")


sample_outputs = model2.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=True,
    max_length=200,
    top_k=0,
    top_p=0.9,
    temperature=1.0,
    num_return_sequences=1
)

print("Output:", tokenizer2.decode(sample_outputs[0], skip_special_tokens=True))

Output: Write verses in a calm and beautiful pastoral style, describing a peaceful spring morning in the countryside. Focus on the beauty of nature—flowers blooming, birds singing, and the gentle breeze. Use soft, vivid language to capture the tranquility of the scene. Each verse should evoke a sense of peace and harmony with the natural world.

Write the poem, generate the verses:

and after a word, each moment is sanctified, or a tale from the wagons.

strike stiffen and noisy admiring the future comes in the spring of the after the low of a sea.


hang on a weird act of a singing after the sky: orchard, orchard, and yawn at least come on a cry in a larger farthest covetendomic orchard, orchard, or night orchard



time in the blue hills, and orchard, and hold it always a timbreasts like a sort of vial


c1: al doilea text ii mai profund si se potriveste mai bine cu primul vers <br>
la primul, trebuie sa setez temperature mai mica ca sa-mi genereze bine, in schimb la al doilea merge cu 1.0 <br>
c2: imi genereaza versuri normal, in limba engleza <br>
c3: primul LLM: dureaza mai mult, mi-a repetat de mai multe ori un vers <br>
    al doilea LLM: primul vers e in romana, si celelalte in engleza <br>
c.4 pentru primul LLM imi genereaza in limba romana <br>
    al doilea LLM imi genereaza in limba engleza <br>
c.5: sa-i dau un prompt mai lung si mai specific sau sa antrenam model pe poezii de tip pastel